# Testing Literature Survey 1 - Social Media

In this notebook, we run our project using a dataset of Youtube Comments acquired through Kaggle. We found this dataset to be very similar to the one used in our literature survey. By testing our models, we can draw a comparison between our performance and that of Naive Bayes reported in the research papers.

## Loading Dataset and Libraries ##

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import string
import os

import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import folium
from itertools import cycle, islice
from pandas import options
import warnings
import pickle
import nltk
from matplotlib.pyplot import figure
from nltk.corpus import stopwords 
import nltk


nltk.download('wordnet')
nltk.download('stopwords')
from tqdm import tqdm,tqdm_notebook

from  wordcloud import WordCloud
%matplotlib inline

[nltk_data] Downloading package wordnet to C:\Users\Abhinav
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Abhinav
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# DATA_SET = './datasets/aclimdb.csv'
DATA_SET = './YoutubeComments.csv'
df =pd.read_csv(DATA_SET)

In [3]:
df.review = df.review.astype(str)
df

,review,sentiment
0,Let's not forget that Apple Pay in 2014 requir...,1
1,Here in NZ 50% of retailers don’t even have co...,0
2,I will forever acknowledge this channel with t...,2
3,Whenever I go to a place that doesn’t take App...,0
4,"Apple Pay is so convenient, secure, and easy t...",2
...,...,...
18404,I really like the point about engineering tool...,2
18405,I’ve just started exploring this field. And th...,2
18406,Excelente video con una pregunta filosófica pr...,1
18407,"Hey Daniel, just discovered your channel a cou...",2


In [4]:
df.sentiment.replace(2, 1, inplace=True)

## Preprocessing Data ##

In [8]:
#lowercasing all the words in the review
df['review']=df['review'].apply(lambda x: x.lower())
df.head()

,review,sentiment
0,let's not forget that apple pay in 2014 requir...,1
1,here in nz 50% of retailers don’t even have co...,0
2,i will forever acknowledge this channel with t...,1
3,whenever i go to a place that doesn’t take app...,0
4,"apple pay is so convenient, secure, and easy t...",1


In [9]:
#contraction to expansion : 
#converting the words in their contracted form to their extracted form eg. he'll to he will
#using the cont_to_exp() and a dictionary:{key: contractions,value:expansion}
contractions = { 
"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not",
"hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not",
"he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how does",
"i'd": "i would","i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have","isn't": "is not",
"it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is","let's": "let us",
"ma'am": "madam","mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have",
"must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have",
"o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have",
"shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
"she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
"should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
"so've": "so have","so's": "so is",
"that'd": "that would","that'd've": "that would have","that's": "that is",
"there'd": "there would","there'd've": "there would have","there's": "there is",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
"to've": "to have","wasn't": "was not",
" u ": " you "," ur ": " your "," n ": " and ",
"ain't": "is not", "aren't": "are not","can't": "cannot", "cause": "because", "could've": "could have", "couldn't": "could not",
"didn't": "did not",  "doesn't": "does not", "don't": "do not", 
"hadn't": "had not", "hasn't": "has not", "haven't": "have not",
"he'd": "he would","he'll": "he will", "he's": "he is", 
"how'd": "how did", "how'd'y": "how do you", 
"how'll": "how will", "how's": "how is",
"I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
"i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
"mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
"mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
"she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
"should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
"this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
"they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
"we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
"what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
"where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"}
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key,value)
        return x
    else : 
        return x
df['review'] = df['review'].apply(lambda x:cont_to_exp(x))


In [10]:
# Removing the urls from the reviews
df['review']=df['review'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',x)) 

In [11]:
#Stopword removal : Stopwords are the words that appear quite frequently in a sentence and do not have a significant contribution to the meaning of the sentence. Therefore they can be removed.
df['review'] = df['review'].apply(lambda x:" ".join([t for t in x.split() if t not in STOP_WORDS ]))


In [12]:
# Removal of special characters from the reviews
df['review']=df['review'].apply(lambda x:re.sub(r'[^0-9a-zA-Z *]','',x))
df['review']=df['review'].apply(lambda x:re.sub(r'[^a-zA-z0-9\s]','',x))

df['review'] = df['review'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))


In [13]:
#Removal of mulitple spaces between the words in the review
df["review"]=df["review"].apply(lambda x: " ".join(x.split()))


In [14]:
#Removal of HTML Tags: from the reviews
df['review'] = df['review'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())

#Remove tags and links 
tag = re.compile(r'<[^>]+>')

df['review'] = df['review'].apply(lambda x: tag.sub('', x)) #removing html labels

df['review'] = df['review'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)

df

,review,sentiment
0,let forget apple pay 2014 required brand new i...,1
1,nz 50 retailers dont contactless credit card m...,0
2,forever acknowledge channel help lessons ideas...,1
3,place doesnt apple pay doesnt happen often its...,0
4,apple pay convenient secure easy use korean ja...,1
...,...,...
18404,like point engineering toolboxes think thats l...,1
18405,ive started exploring field good reminder earl...,1
18406,excelente video con una pregunta filosfica pro...,1
18407,hey daniel discovered channel couple days ago ...,1


In [15]:
# Removal of Numbers
df['review']=df['review'].apply(lambda x:re.sub(r'[0-9]+','',x))

In [16]:
# Removal of usernames from the reviews
df['review']=df['review'].apply(lambda x:re.sub(r'@[A-Za-z0–9]+','',x))
df.sample(10)

,review,sentiment
9593,mee goreng vibe definitely childhood faves awe...,1
13362,hybrid warfare is terrorism simple plain cowar...,0
15610,st console original ps loved stayed upgraded p...,1
2319,spot maintenance number owned trucking compan...,1
13228,wish guests answer questionsand points,1
4945,excited believe focused iphone apple watch may...,1
285,said packaging affecting taste true think deli...,1
12114,im worried cant sat august tt techniques please,0
10998,a vida to bela alis o nosso planeta terra tem ...,1
10100,numbers screen climbing gforce g stands gravi...,1


In [17]:
# Tokenization and Lemmanization

nltk.download('omw-1.4')

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df['review'] = df.review.apply(lemmatize_text).copy()

df

[nltk_data] Downloading package omw-1.4 to C:\Users\Abhinav
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,review,sentiment
0,let forget apple pay require brand new iphone ...,1
1,nz retailers dont contactless credit card mach...,0
2,forever acknowledge channel help lessons ideas...,1
3,place doesnt apple pay doesnt happen often its...,0
4,apple pay convenient secure easy use korean ja...,1
...,...,...
18404,like point engineer toolboxes think thats lot ...,1
18405,ive start explore field good reminder early jo...,1
18406,excelente video con una pregunta filosfica pro...,1
18407,hey daniel discover channel couple days ago le...,1


## Splitting Data ##

In [22]:
# Splitting Dataset into training and testing sets
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score

x = pd.DataFrame(df, columns = ['review']) 
y = pd.DataFrame(df, columns = ['sentiment']) 

# Split dataset to train and test set.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

print("Shape of x_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of x_test:  ", X_test.shape)
print("Shape of y_test:  ", y_test.shape)

Shape of x_train:  (13806, 1)
Shape of y_train:  (13806, 1)
Shape of x_test:   (4603, 1)
Shape of y_test:   (4603, 1)


## Vectorization with TF-IDF ##

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

train = X_train['review'].tolist()
test = X_test['review'].tolist()
STOPWORDS = set(stopwords.words('english'))

tfidf_vectorizer = TfidfVectorizer(max_features=1000, dtype=np.float32)

tfidfX_train = tfidf_vectorizer.fit_transform(train)
tfidfX_train = tfidfX_train.toarray()

tfidfX_test = tfidf_vectorizer.transform(test)
tfidfX_test = tfidfX_test.toarray()

print("TF-IDF train shape:", tfidfX_train.shape)
print("TF-IDF test shape:", tfidfX_test.shape)

TF-IDF train shape: (13806, 1000)
TF-IDF test shape: (4603, 1000)


## Decision Tree ##

In [27]:
from sklearn.tree import DecisionTreeClassifier

dct = DecisionTreeClassifier(criterion='entropy', random_state=1)
dct.fit(tfidfX_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=1)

In [28]:
# Making Predictions
y_pred_dct = dct.predict(tfidfX_test)

# Evaluation Metrics

dct_accuracy = accuracy_score(y_test,y_pred_dct)*100
dct_matrix = confusion_matrix(y_test,y_pred_dct)
dct_precision = dct_matrix[0][0]*100/(dct_matrix[0][0]+dct_matrix[1][0])
dct_recall = dct_matrix[0][0]*100/(dct_matrix[0][0]+dct_matrix[0][1])

print("Accuracy : ",dct_accuracy)
print("Confusion_matrix:\n",dct_matrix)
# print("precision:",dct_precision)
# print("recall:",dct_recall)

Accuracy :  83.38040408429285
Confusion_matrix:
 [[ 179  417]
 [ 348 3659]]


## XG Boosting ##

In [29]:
from xgboost import XGBClassifier

xg = XGBClassifier(random_state=22,learning_rate=0.9)
xg.fit(tfidfX_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.9, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=22, ...)

In [30]:
# Making Predictions
y_pred_xg = xg.predict(tfidfX_test)

# Evaluation Metrics

xg_accuracy = accuracy_score(y_test,y_pred_xg)*100
xg_matrix = confusion_matrix(y_test,y_pred_xg)
xg_precision = xg_matrix[0][0]*100/(xg_matrix[0][0]+xg_matrix[1][0])
xg_recall = xg_matrix[0][0]*100/(xg_matrix[0][0]+xg_matrix[0][1])

print("Accuracy : ",xg_accuracy)
print("Confusion_matrix:\n",xg_matrix)
# print("precision:",xg_precision)
# print("recall:",xg_recall)

Accuracy :  86.8563980013035
Confusion_matrix:
 [[ 161  435]
 [ 170 3837]]


## Random Forest ##

In [31]:
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(n_estimators=150,max_depth=None)
rf1.fit(tfidfX_train,y_train)

C:\Users\ABHINA~1\AppData\Local\Temp/ipykernel_12116/2677021949.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(tfidfX_train,y_train)


RandomForestClassifier(n_estimators=150)

In [32]:
# Making Predictions
y_pred_rf1 = rf1.predict(tfidfX_test)

# Evaluation Metrics

rf1_accuracy = accuracy_score(y_test,y_pred_rf1)*100
rf1_matrix = confusion_matrix(y_test,y_pred_rf1)
rf1_precision = rf1_matrix[0][0]*100/(rf1_matrix[0][0]+rf1_matrix[1][0])
rf1_recall = rf1_matrix[0][0]*100/(rf1_matrix[0][0]+rf1_matrix[0][1])

print("Accuracy : ",rf1_accuracy)
print("Confusion_matrix:\n",rf1_matrix)
# print("precision:",rf1_precision)
# print("recall:",rf1_recall)

Accuracy :  87.46469693678036
Confusion_matrix:
 [[  73  523]
 [  54 3953]]


## Logistic Regression ##

In [33]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0,solver='lbfgs')

lr.fit(tfidfX_train,y_train)

C:\Softwares\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(random_state=0)

In [34]:
# Making Predictions
y_pred_lr = lr.predict(tfidfX_test)

# Evaluation Metrics

lr_accuracy = accuracy_score(y_test,y_pred_lr)*100
lr_matrix = confusion_matrix(y_test,y_pred_lr)
lr_precision = lr_matrix[0][0]*100/(lr_matrix[0][0]+lr_matrix[1][0])
lr_recall = lr_matrix[0][0]*100/(lr_matrix[0][0]+lr_matrix[0][1])

print("Accuracy : ",lr_accuracy)
print("Confusion_matrix:\n",lr_matrix)
# print("precision:",lr_precision)
# print("recall:",lr_recall)

Accuracy :  87.94264610036933
Confusion_matrix:
 [[  90  506]
 [  49 3958]]


## Extra Tree Classifier ##

In [35]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(random_state=123)
etc.fit(tfidfX_train,y_train)

C:\Users\ABHINA~1\AppData\Local\Temp/ipykernel_12116/165346209.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  etc.fit(tfidfX_train,y_train)


ExtraTreesClassifier(random_state=123)

In [36]:
# Making Predictions
y_pred_etc = etc.predict(tfidfX_test)

# Evaluation Metrics

etc_accuracy = accuracy_score(y_test,y_pred_etc)*100
etc_matrix = confusion_matrix(y_test,y_pred_etc)
etc_precision = etc_matrix[0][0]*100/(etc_matrix[0][0]+etc_matrix[1][0])
etc_recall = etc_matrix[0][0]*100/(etc_matrix[0][0]+etc_matrix[0][1])

print("Accuracy : ",etc_accuracy)
print("Confusion_matrix:\n",etc_matrix)
# print("precision:",etc_precision)
# print("recall:",etc_recall)

Accuracy :  87.26917227894852
Confusion_matrix:
 [[  69  527]
 [  59 3948]]


## Voting ##

In [37]:
# Defining Estimators

estimators = [('dct',dct),('xg',xg),('rf1',rf1),('lr',lr),('etc',etc)]
estimators

[('dct', DecisionTreeClassifier(criterion='entropy', random_state=1)),
 ('xg',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.9, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=None, num_parallel_tree=None,
                predictor=None, random_state=22, ...)),
 ('rf1', RandomForestClassifier(n_estimators=150)),
 ('lr', LogisticRegression(random_state=0)),
 ('etc', ExtraTreesClassifier(random_state=123))]

In [38]:
# Evaluating the Estimator

from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators, weights=[0.5,1,2.5,1,2.5])
vc.fit(tfidfX_train,y_train)

C:\Softwares\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


VotingClassifier(estimators=[('dct',
                              DecisionTreeClassifier(criterion='entropy',
                                                     random_state=1)),
                             ('xg',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=...
                                            max_delta_step=None, max_depth=None,
           

In [39]:
# Making Predictions
y_pred_vc = vc.predict(tfidfX_test)

# Evaluation Metrics

vc_accuracy = accuracy_score(y_test,y_pred_vc)*100
vc_matrix = confusion_matrix(y_test,y_pred_vc)
vc_precision = vc_matrix[0][0]*100/(vc_matrix[0][0]+vc_matrix[1][0])
vc_recall = vc_matrix[0][0]*100/(vc_matrix[0][0]+vc_matrix[0][1])

print("Accuracy : ",vc_accuracy)
print("Confusion_matrix:\n",vc_matrix)
# print("precision:",vc_precision)
# print("recall:",vc_recall)

Accuracy :  87.59504670866825
Confusion_matrix:
 [[  75  521]
 [  50 3957]]


In [40]:
print("Testing Accuracies")
acc_list = {
    'Decision Tree':dct_accuracy,
    'XG':xg_accuracy,
    'Random Forest':rf1_accuracy,
    'Logistic Regression':lr_accuracy,
    'Extra Tree Classifier':etc_accuracy,
    'Voting Classifier':vc_accuracy
}
acc_df_test = pd.DataFrame.from_dict(acc_list,orient="index",columns=['Accuracy'])
acc_df_test

Testing Accuracies


,Accuracy
Decision Tree,83.380404
XG,86.856398
Random Forest,87.464697
Logistic Regression,87.942646
Extra Tree Classifier,87.269172
Voting Classifier,87.595047
